# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import pandas as pd
from sqlalchemy import create_engine
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
#classifiers
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [2]:
# load data from database
engine = create_engine('sqlite:///message.db')
df = pd.read_sql_table('messages',engine)
df.shape

(26345, 40)

In [3]:
#drop duplicates
df.drop_duplicates(subset='message', keep='first', inplace=True)

In [4]:
#find invalid classifications and drop them
df=df[df[df.columns[4]]!=2]

In [5]:
#drop features with only '0' which are unused classes
df.drop(columns=['child_alone'], inplace=True)

In [6]:
#drop rows with only '0' which are not classified
df=df.loc[df[df.columns[4:40]].sum(axis=1)!=0].reset_index(drop=True)

In [7]:
df.shape

(19763, 39)

In [8]:
#prepare feature and classes
X = df['message'].values
Y = df[df.columns[4:40]].values

### 2. Write a tokenization function to process your text data

In [9]:
def tokenize(text):
    
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ",text)

    # tokenize text
    tokens = word_tokenize(text)
    
    # Remove stop words
    tokens = [w for w in tokens if w not in stopwords.words("english")]
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
pipeline = Pipeline([
        ('vect',CountVectorizer(tokenizer=tokenize)),
        ('tfidf',TfidfTransformer()),
        ('clf',MultiOutputClassifier(RandomForestClassifier()))
    ])     

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .30, random_state=42)
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000002BD07BCEE50>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
Y_pred=pipeline.predict(X_test)

In [16]:
Y_pred

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [18]:
#transform results into dataframe
Y_pred_df=pd.DataFrame(Y_pred, columns=df.columns[4:40])
Y_pred_df.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [23]:
f1_score=[]
precision=[]
for n in range(4,len(df.columns)):
    try:
        s=classification_report(pd.DataFrame(Y_test, columns=df.columns[4:40])[df.columns[n]], Y_pred_df[[df.columns[n]]],output_dict=True)
        precision.append(s['1.0']['precision'])
        f1_score.append(s['1.0']['f1-score'])
    except:
        print(df.columns[n]+' does not have a 1')
        continue


In [24]:
print('avg_precision:'+str(sum(precision)/len(precision))+', avg_f1-score:'+str(sum(f1_score)/len(f1_score)))

avg_precision:0.17578436888656668, avg_f1-score:0.09797370403745095


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = None

cv = None

In [25]:
#check different classifiers
names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

In [ ]:
#loop through classifier
results=[]
for name, clf in zip(names, classifiers):
    pipeline = Pipeline([
        ('vect',CountVectorizer(tokenizer=tokenize)),
        ('tfidf',TfidfTransformer()),
        ('clf',MultiOutputClassifier(clf))
    ])    
    #split data
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .30, random_state=42)
    #train data and predict test data
    pipeline.fit(X_train)    
    Y_pred=pipeline.predict(X_test)
    
    #transform results into dataframe
    Y_pred_df=pd.DataFrame(Y_pred, columns=df.columns[4:40])
    
    #print and store precision and f1-score
    f1_score=[]
    precision=[]
    for n in range(4,len(df.columns)):
        try:
            s=classification_report(pd.DataFrame(Y_test, columns=df.columns[4:40])[df.columns[n]], Y_pred_df[[df.columns[n]]],output_dict=True)
            precision.append(s['1.0']['precision'])
            f1_score.append(s['1.0']['f1-score'])
        except:
            print(df.columns[n]+' does not have a 1')
            continue
    print('Classifier:'+name+'\navg_precision:'+str(sum(precision)/len(precision))+', avg_f1-score:'+str(sum(f1_score)/len(f1_score)))
    
    results.append([name, sum(precision)/len(precision), sum(f1_score)/len(f1_score)])

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.